# Load Huggingface Pytorch Fill Mask Bert Model

In [1]:
import $ivy.`ai.djl:api:0.17.0`
import $ivy.`ai.djl.huggingface:tokenizers:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-model-zoo:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-engine:0.17.0`
import $ivy.`org.slf4j:slf4j-api:1.7.36`
import $ivy.`org.slf4j:slf4j-simple:1.7.36`
import $ivy.`com.lihaoyi::upickle:0.9.5`

Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle_2.12/0.9.5/upickle_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upack_2.12/0.9.5/upack_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-implicits_2.12/0.9.5/upickle-implicits_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/ujson_2.12/0.9.5/ujson_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle_2.12/0.9.5/upickle_2.12-0.9.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-implicits_2.12/0.9.5/upickle-implicits_2.12-0.9.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/liha

import $ivy.$                  

import $ivy.$                                     

import $ivy.$                                        

import $ivy.$                                     

import $ivy.$                           

import $ivy.$                              

import $ivy.$                           

In [2]:
import java.io.IOException
import java.nio.file.{Files, Paths}
import java.util

import ai.djl.modality.Classifications
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.bert.BertTokenizer
import ai.djl.modality.nlp.Vocabulary
import ai.djl.ndarray.NDList
import ai.djl.repository.zoo.Criteria
import ai.djl.training.util.{DownloadUtils, ProgressBar}
import ai.djl.translate.{Batchifier, Translator, TranslatorContext}
import ai.djl.huggingface.tokenizers.HuggingFaceTokenizer

import java.io.IOException

import java.nio.file.{Files, Paths}

import java.util


import ai.djl.modality.Classifications

import ai.djl.modality.nlp.DefaultVocabulary

import ai.djl.modality.nlp.bert.BertTokenizer

import ai.djl.modality.nlp.Vocabulary

import ai.djl.ndarray.NDList

import ai.djl.repository.zoo.Criteria

import ai.djl.training.util.{DownloadUtils, ProgressBar}

import ai.djl.translate.{Batchifier, Translator, TranslatorContext}

import ai.djl.huggingface.tokenizers.HuggingFaceTokenizer

In [3]:
case class PredictedLabel(label: String, score: Double)

defined class PredictedLabel

In [4]:
class HFZeroShotClsTranslator extends Translator[String, Seq[PredictedLabel]] {

  private var vocabulary: DefaultVocabulary = _
  private var tokenizer: HuggingFaceTokenizer = _
  private var tokenList: Array[String] = _

  override def prepare(ctx: TranslatorContext): Unit = {
    val path = Paths.get("../build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.txt")
    vocabulary = DefaultVocabulary.builder
      .optMinFrequency(1)
      .addFromTextFile(path)
      .optUnknownToken("<unk>")
      .build
    tokenizer = HuggingFaceTokenizer.newInstance("facebook/bart-large-mnli")
  }

  override def processInput(ctx: TranslatorContext, input: String): NDList = {
    val token = tokenizer.encode(input.toLowerCase().replace(MaskToken.toLowerCase(), MaskToken))
    // get the encoded tokens that would be used in precessOutput
    tokenList = token.getTokens
    // map the tokens(String) to indices(long)

    val manager = ctx.getNDManager
    val indices = tokenList.map(vocabulary.getIndex)
    val attentionMask = token.getAttentionMask.map(i => i)
    val indicesArray = manager.create(indices)
    val attentionMaskArray = manager.create(attentionMask)

    new NDList(indicesArray, attentionMaskArray)
  }

  override def processOutput(ctx: TranslatorContext, list: NDList): Seq[PredictedToken] = {
    val maskIndex = tokenList.zipWithIndex.find(_._1 == MaskToken).map(_._2).getOrElse(-1)
    if (maskIndex == -1) {
      Seq.empty[PredictedToken]
    } else {
      val ndArray = list.get(0)
      val shape = ndArray.getShape
      val len = shape.get(1)

      (1 to TopK).map { i =>
        val out = ndArray.get(maskIndex).argSort().getLong(len - i)
        PredictedToken(vocabulary.getToken(out), ndArray.getFloat(maskIndex, out))
      }
    }
  }

  override def getBatchifier: Batchifier = Batchifier.STACK
}

defined class HFBertFillMaskTranslator

In [5]:
val input = "Paris is the [MASK] of France.".toLowerCase.replace("[mask]", "[MASK]")

val translator = new HFBertFillMaskTranslator()
val criteria = Criteria.builder
  .setTypes(classOf[String], classOf[Seq[PredictedToken]])
  .optModelPath(Paths.get("../build/huggingface/fill_mask/pytorch/bert-base-uncased/"))
  .optTranslator(translator)
  .optProgress(new ProgressBar)
  .build

val model = criteria.loadModel()

val predictor = model.newPredictor(translator)

val predictResult = predictor.predict(input)
predictResult.foreach(println(_))

Loading:     100% |████████████████████████████████████████|


[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of inter-op threads is 4
[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of intra-op threads is 4
[scala-interpreter-1] INFO ai.djl.huggingface.tokenizers.jni.LibUtils - Extracting native/lib/osx-x86_64/libtokenizers.dylib to cache ...


PredictedToken(capital,18.19974136352539)
PredictedToken(heart,10.769936561584473)
PredictedToken(center,10.469231605529785)
PredictedToken(centre,10.209856986999512)
PredictedToken(city,9.985564231872559)


input: String = "paris is the [MASK] of france."
translator: HFBertFillMaskTranslator = ammonite.$sess.cmd3$Helper$HFBertFillMaskTranslator@241526f0
criteria: Criteria[String, Seq[PredictedToken]] = Criteria:
	Application: UNDEFINED
	Input: class java.lang.String
	Output: interface scala.collection.Seq
	ModelZoo: ai.djl.localmodelzoo

model: ai.djl.repository.zoo.ZooModel[String, Seq[PredictedToken]] = ai.djl.repository.zoo.ZooModel@6b002e0e
predictor: ai.djl.inference.Predictor[String, Seq[PredictedToken]] = ai.djl.inference.Predictor@c132fed
predictResult: Seq[PredictedToken] = Vector(
  PredictedToken("capital", 18.19974136352539),
  PredictedToken("heart", 10.769936561584473),
  PredictedToken("center", 10.469231605529785),
  PredictedToken("centre", 10.209856986999512),
  PredictedToken("city", 9.985564231872559)
)

In [10]:
val encoded = Seq( 0, 10285,   186,    38,  7084,   127,  6894,  1732,     8,   655,
           187,   172,   127,  1028,    34,    57, 27169,  1295,  8378,    38,
           304,   110,  1553,     4,     2,     2,   713,  1246,    16,  1830,
             4,     2)

encoded: Seq[Int] = List(
  0,
  10285,
  186,
  38,
  7084,
  127,
  6894,
  1732,
  8,
  655,
  187,
  172,
  127,
  1028,
  34,
  57,
  27169,
  1295,
  8378,
  38,
  304,
  110,
  1553,
  4,
  2,
  2,
  713,
  1246,
  16,
  1830,
  4,
  2
)

In [11]:
val path = Paths.get("../build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.txt")
val vocabulary = DefaultVocabulary.builder
      .optMinFrequency(1)
      .addFromTextFile(path)
      .optUnknownToken("<unk>")
      .build

path: java.nio.file.Path = ../build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.txt
vocabulary: DefaultVocabulary = ai.djl.modality.nlp.DefaultVocabulary@5f16d5d7

In [12]:
encoded.foreach(r => println(vocabulary.getToken(r)))

<s>
Last
Ġweek
ĠI
Ġupgraded
Ġmy
ĠiOS
Ġversion
Ġand
Ġever
Ġsince
Ġthen
Ġmy
Ġphone
Ġhas
Ġbeen
Ġoverhe
ating
Ġwhenever
ĠI
Ġuse
Ġyour
Ġapp
.
</s>
</s>
This
Ġexample
Ġis
Ġmobile
.
</s>


In [13]:
val tokenizer = HuggingFaceTokenizer.newInstance("facebook/bart-large-mnli")

tokenizer: HuggingFaceTokenizer = ai.djl.huggingface.tokenizers.HuggingFaceTokenizer@7d7db9a3

In [21]:
val premise = "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
val labels = Array("mobile", "website", "billing", "account access")
val hypothesis = s"This example is ${labels(0)}."
tokenizer.encode(Array(premise, "</s>", "</s>", hypothesis)).getTokens

premise: String = "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
labels: Array[String] = Array("mobile", "website", "billing", "account access")
hypothesis: String = "This example is mobile."
res20_3: Array[String] = Array(
  "<s>",
  "Last",
  "\u0120week",
  "\u0120I",
  "\u0120upgraded",
  "\u0120my",
  "\u0120iOS",
  "\u0120version",
  "\u0120and",
  "\u0120ever",
  "\u0120since",
  "\u0120then",
  "\u0120my",
  "\u0120phone",
  "\u0120has",
  "\u0120been",
  "\u0120overhe",
  "ating",
  "\u0120whenever",
  "\u0120I",
  "\u0120use",
  "\u0120your",
  "\u0120app",
  ".",
  "</s>",
  "</s>",
  "This",
  "\u0120example",
  "\u0120is",
  "\u0120mobile",
  ".",
  "</s>"
)

In [22]:
encoded.foreach(r => println(vocabulary.getToken(r)))

<s>
Last
Ġweek
ĠI
Ġupgraded
Ġmy
ĠiOS
Ġversion
Ġand
Ġever
Ġsince
Ġthen
Ġmy
Ġphone
Ġhas
Ġbeen
Ġoverhe
ating
Ġwhenever
ĠI
Ġuse
Ġyour
Ġapp
.
</s>
</s>
This
Ġexample
Ġis
Ġmobile
.
</s>
